In [11]:
import itertools

# Let's lay out the 5 properties to be assigned to each of the people in the 5 houses.
#addresses = [1,2,3,4,5] # Not needed; implicit in tuple indices of permutations.
nationalities = ["Brit", "Swede", "Dane", "Norwegian", "German"]
colors = ["red", "green", "white", "yellow", "blue"]
drinks = ["tea", "coffee", "milk", "beer", "water"]
pets = ["dog", "bird", "cat", "horse", "fish"]
cigars = ["Pall Mall", "Dunhill", "Blend", "Bluemaster", "Prince"]



Let's start with a brute-force solution:

In [9]:
for nationality_order in itertools.permutations(nationalities, 5):
    print(nationality_order)

('Brit', 'Swede', 'Dane', 'Norwegian', 'German')
('Brit', 'Swede', 'Dane', 'German', 'Norwegian')
('Brit', 'Swede', 'Norwegian', 'Dane', 'German')
('Brit', 'Swede', 'Norwegian', 'German', 'Dane')
('Brit', 'Swede', 'German', 'Dane', 'Norwegian')
('Brit', 'Swede', 'German', 'Norwegian', 'Dane')
('Brit', 'Dane', 'Swede', 'Norwegian', 'German')
('Brit', 'Dane', 'Swede', 'German', 'Norwegian')
('Brit', 'Dane', 'Norwegian', 'Swede', 'German')
('Brit', 'Dane', 'Norwegian', 'German', 'Swede')
('Brit', 'Dane', 'German', 'Swede', 'Norwegian')
('Brit', 'Dane', 'German', 'Norwegian', 'Swede')
('Brit', 'Norwegian', 'Swede', 'Dane', 'German')
('Brit', 'Norwegian', 'Swede', 'German', 'Dane')
('Brit', 'Norwegian', 'Dane', 'Swede', 'German')
('Brit', 'Norwegian', 'Dane', 'German', 'Swede')
('Brit', 'Norwegian', 'German', 'Swede', 'Dane')
('Brit', 'Norwegian', 'German', 'Dane', 'Swede')
('Brit', 'German', 'Swede', 'Dane', 'Norwegian')
('Brit', 'German', 'Swede', 'Norwegian', 'Dane')
('Brit', 'German', '

In [17]:
from tqdm import tqdm

In [ ]:
for assignment in tqdm(itertools.product(
    itertools.permutations(nationalities, 5), # 0
    itertools.permutations(colors, 5), # 1
    itertools.permutations(drinks, 5), # 2
    itertools.permutations(pets, 5), # 3
    itertools.permutations(cigars, 5))): # 4

    # These two conditions are fast to check.
    if assignment[2][2] != "milk":
        continue # 8. The man living in the center house drinks milk.
    if assignment[0][0] != "Norwegian":
        continue # 9. The Norwegian lives in the first house.
    
    if assignment[0].index("Brit") != assignment[1].index("red"):
        continue # 1. The Brit lives in the red house.
    if assignment[0].index("Swede") != assignment[3].index("dog"):
        continue # 2. The Swede keeps dogs as pets.
    if assignment[0].index("Dane") != assignment[2].index("tea"):
        continue # 3. The Dane drinks tea.
    if assignment[1].index("green") != assignment[1].index("white")-1:
        continue # 4. The green house is to the left of the white house.
    if assignment[1].index("green") != assignment[2].index("coffee"):
        continue # 5. The green homeowner drinks coffee.
    if assignment[4].index("Pall Mall") != assignment[3].index("bird"):
        continue # 6. The person who smokes Pall Mall rears birds.
    if assignment[1].index("yellow") != assignment[4].index("Dunhill"):
        continue # 7. The owner of the yellow house smokes Dunhill.
    
    # Conditions 8 and 9 moved to top for faster possibility elimination
    
    if abs(assignment[4].index("Blend") - assignment[3].index("cat")) != 1:
        continue # 10. The man who smokes Blend lives next to the one who keeps cats.
    if abs(assignment[4].index("Dunhill") - assignment[3].index("horse")) != 1:
        continue # 11. The man who keeps horses lives next to the man who smokes Dunhill.
    if assignment[4].index("Bluemaster") != assignment[2].index("beer"):
        continue # 12. The owner who smokes Bluemaster drinks beer.
    if assignment[0].index("German") != assignment[4].index("Prince"):
        continue # 13. The German smokes Prince.
    if abs(assignment[0].index("Norwegian") - assignment[1].index("blue")) != 1:
        continue # 14. The Norwegian lives next to the blue house.
    if abs(assignment[4].index("Blend") - assignment[2].index("water")) != 1:
        continue # 15. The man who smokes Blend has a neighbor who drinks water.
    
    print(assignment)

140760153it [00:59, 2525810.77it/s]

In [14]:
assignment

(('Brit', 'Swede', 'Dane', 'German', 'Norwegian'),
 ('white', 'blue', 'red', 'green', 'yellow'),
 ('beer', 'coffee', 'tea', 'milk', 'water'),
 ('dog', 'bird', 'cat', 'fish', 'horse'),
 ('Bluemaster', 'Dunhill', 'Pall Mall', 'Prince', 'Blend'))

Unfortunately, even running at roughly 2 million iterations per second, we still are iterating over $120^5$ = 24,883,200,000 possibilities, taking roughly 12,000 seconds, or over 3 hours. (And that's an optimistic estimate, since the first assigments checked all have the Brit in the first house, failing quickly on only the second condition. Things will get slower when getting into the possibilities that pass the first couple of fast checks and go into the slower conditions that require indexing.) This is ... bad. Thus is the power of exponentiation.

The code is fairly dumb in that, after eliminating a possibility for having the milk-drinker in the wrong place, it will continue to loop over and check every other assigment possibility with the same permutation of beverages. Why dont we just skip forward to assignments that have different arrangements of drinks, and the same for nationalities? This should cut down on the work by a factor of about 25, focusing only on the assignments with the Norwegian in the first house and the milk-drinker in the third.

In [6]:
import time
start_time = time.time()

for n_perm in itertools.permutations(nationalities, 5):
    print(n_perm)
    if n_perm[0] != "Norwegian":
        continue # 9. The Norwegian lives in the first house.
        
    for d_perm in itertools.permutations(drinks, 5):
        if d_perm[2] != "milk":
            continue # 8. The man living in the center house drinks milk.
        
        for col_perm in itertools.permutations(colors, 5):
            for p_perm in itertools.permutations(pets, 5):
                for cig_perm in itertools.permutations(cigars, 5):
                    
                    if n_perm.index("Brit") != col_perm.index("red"):
                        continue # 1. The Brit lives in the red house.
                    if n_perm.index("Swede") != p_perm.index("dog"):
                        continue # 2. The Swede keeps dogs as pets.
                    if n_perm.index("Dane") != d_perm.index("tea"):
                        continue # 3. The Dane drinks tea.
                    if col_perm.index("green") != col_perm.index("white")-1:
                        continue # 4. The green house is to the left of the white house.
                    if col_perm.index("green") != d_perm.index("coffee"):
                        continue # 5. The green homeowner drinks coffee.
                    if cig_perm.index("Pall Mall") != p_perm.index("bird"):
                        continue # 6. The person who smokes Pall Mall rears birds.
                    if col_perm.index("yellow") != cig_perm.index("Dunhill"):
                        continue # 7. The owner of the yellow house smokes Dunhill.
                    
                    # Conditions 8 and 9 moved to top for faster possibility elimination
                    
                    if abs(cig_perm.index("Blend") - p_perm.index("cat")) != 1:
                        continue # 10. The man who smokes Blend lives next to the one who keeps cats.
                    if abs(cig_perm.index("Dunhill") - p_perm.index("horse")) != 1:
                        continue # 11. The man who keeps horses lives next to the man who smokes Dunhill.
                    if cig_perm.index("Bluemaster") != d_perm.index("beer"):
                        continue # 12. The owner who smokes Bluemaster drinks beer.
                    if n_perm.index("German") != cig_perm.index("Prince"):
                        continue # 13. The German smokes Prince.
                    if abs(n_perm.index("Norwegian") - col_perm.index("blue")) != 1:
                        continue # 14. The Norwegian lives next to the blue house.
                    if abs(cig_perm.index("Blend") - d_perm.index("water")) != 1:
                        continue # 15. The man who smokes Blend has a neighbor who drinks water.
                    
                    print("Solution found!")
                    print(col_perm, n_perm, d_perm, cig_perm, p_perm)
                    print(f"Elapsed time: {time.time()- start_time} seconds")
print(f"Total time: {time.time()- start_time} seconds")

('Brit', 'Swede', 'Dane', 'Norwegian', 'German')
('Brit', 'Swede', 'Dane', 'German', 'Norwegian')
('Brit', 'Swede', 'Norwegian', 'Dane', 'German')
('Brit', 'Swede', 'Norwegian', 'German', 'Dane')
('Brit', 'Swede', 'German', 'Dane', 'Norwegian')
('Brit', 'Swede', 'German', 'Norwegian', 'Dane')
('Brit', 'Dane', 'Swede', 'Norwegian', 'German')
('Brit', 'Dane', 'Swede', 'German', 'Norwegian')
('Brit', 'Dane', 'Norwegian', 'Swede', 'German')
('Brit', 'Dane', 'Norwegian', 'German', 'Swede')
('Brit', 'Dane', 'German', 'Swede', 'Norwegian')
('Brit', 'Dane', 'German', 'Norwegian', 'Swede')
('Brit', 'Norwegian', 'Swede', 'Dane', 'German')
('Brit', 'Norwegian', 'Swede', 'German', 'Dane')
('Brit', 'Norwegian', 'Dane', 'Swede', 'German')
('Brit', 'Norwegian', 'Dane', 'German', 'Swede')
('Brit', 'Norwegian', 'German', 'Swede', 'Dane')
('Brit', 'Norwegian', 'German', 'Dane', 'Swede')
('Brit', 'German', 'Swede', 'Dane', 'Norwegian')
('Brit', 'German', 'Swede', 'Norwegian', 'Dane')
('Brit', 'German', '

In [7]:
print(f"Total time: {time.time()- start_time} seconds")

Total time: 380.4516410827637 seconds


That's probably the first reasonable-ish solution, though it still has to check about a billion possibilities in more or less brute-force fashion. The next step up is probably to try to implement a backtracking algorithm, which is already there to some extent in the quick checking of the beverage and nationality.

Actually, I lied. Let's squeeze a little more performance out of this approach (move on early; break out of a loop and forego the inner loops if an outer loop condition is failing) by systematically moving every condition to the outermost layer it can be in. This might not be the fastest ordering of the 5 nested loops (yeah, that's bad, haha), but while I can imagine doing some systematic search over all 5! = 120 orderings, I'm not doing it (yet?).

In [15]:
import time
start_time = time.time()
#iter_count = 0

for n_perm in itertools.permutations(nationalities, 5):
    print(n_perm)
    if n_perm[0] != "Norwegian":
        continue # 9. The Norwegian lives in the first house.
        
    for d_perm in itertools.permutations(drinks, 5):
        if d_perm[2] != "milk":
            continue # 8. The man living in the center house drinks milk.
        if n_perm.index("Dane") != d_perm.index("tea"):
            continue # 3. The Dane drinks tea.
        
        for col_perm in itertools.permutations(colors, 5):
            if col_perm.index("green") != col_perm.index("white")-1:
                continue # 4. The green house is to the left of the white house.
            if n_perm.index("Brit") != col_perm.index("red"):
                continue # 1. The Brit lives in the red house.
            if col_perm.index("green") != d_perm.index("coffee"):
                continue # 5. The green homeowner drinks coffee.
            if abs(n_perm.index("Norwegian") - col_perm.index("blue")) != 1:
                continue # 14. The Norwegian lives next to the blue house.
                                    
            for p_perm in itertools.permutations(pets, 5):
                if n_perm.index("Swede") != p_perm.index("dog"):
                    continue # 2. The Swede keeps dogs as pets.
                    
                for cig_perm in itertools.permutations(cigars, 5):
                    #iter_count += 1
                    if cig_perm.index("Pall Mall") != p_perm.index("bird"):
                        continue # 6. The person who smokes Pall Mall rears birds.
                    if col_perm.index("yellow") != cig_perm.index("Dunhill"):
                        continue # 7. The owner of the yellow house smokes Dunhill.
 
                    if abs(cig_perm.index("Blend") - p_perm.index("cat")) != 1:
                        continue # 10. The man who smokes Blend lives next to the one who keeps cats.
                    if abs(cig_perm.index("Dunhill") - p_perm.index("horse")) != 1:
                        continue # 11. The man who keeps horses lives next to the man who smokes Dunhill.
                    if cig_perm.index("Bluemaster") != d_perm.index("beer"):
                        continue # 12. The owner who smokes Bluemaster drinks beer.
                    if n_perm.index("German") != cig_perm.index("Prince"):
                        continue # 13. The German smokes Prince.
                    
                    if abs(cig_perm.index("Blend") - d_perm.index("water")) != 1:
                        continue # 15. The man who smokes Blend has a neighbor who drinks water.
                    
                    print("Solution found!")
                    print(col_perm, n_perm, d_perm, cig_perm, p_perm)
                    print(f"Elapsed time: {time.time()- start_time} seconds")
print(f"Total time: {time.time()- start_time} seconds")

('Brit', 'Swede', 'Dane', 'Norwegian', 'German')
('Brit', 'Swede', 'Dane', 'German', 'Norwegian')
('Brit', 'Swede', 'Norwegian', 'Dane', 'German')
('Brit', 'Swede', 'Norwegian', 'German', 'Dane')
('Brit', 'Swede', 'German', 'Dane', 'Norwegian')
('Brit', 'Swede', 'German', 'Norwegian', 'Dane')
('Brit', 'Dane', 'Swede', 'Norwegian', 'German')
('Brit', 'Dane', 'Swede', 'German', 'Norwegian')
('Brit', 'Dane', 'Norwegian', 'Swede', 'German')
('Brit', 'Dane', 'Norwegian', 'German', 'Swede')
('Brit', 'Dane', 'German', 'Swede', 'Norwegian')
('Brit', 'Dane', 'German', 'Norwegian', 'Swede')
('Brit', 'Norwegian', 'Swede', 'Dane', 'German')
('Brit', 'Norwegian', 'Swede', 'German', 'Dane')
('Brit', 'Norwegian', 'Dane', 'Swede', 'German')
('Brit', 'Norwegian', 'Dane', 'German', 'Swede')
('Brit', 'Norwegian', 'German', 'Swede', 'Dane')
('Brit', 'Norwegian', 'German', 'Dane', 'Swede')
('Brit', 'German', 'Swede', 'Dane', 'Norwegian')
('Brit', 'German', 'Swede', 'Norwegian', 'Dane')
('Brit', 'German', '

It turns out that this actually nets us not a *little* more performance, but rather a factor of $213.8/0.01114 \approx 19{,}200$. Wow! Not bad. The layers of early branch pruning means that the innermost loop only iterates a total of 23,040 times (iter_count).

That's perhaps motivation to explore this avenue and play with this technique a little more. Nested loops are frankly a little silly. How about we refactor the loops into functions?

In [13]:
iter_count

23040

In [21]:
(lambda x:x**2)(5)

25

In [18]:
import time
start_time = time.time()
#iter_count = 0

#properties = [nationalities, colors, drinks, pets, cigars]
# properties = {"nation":nationalities,
#               "color":colors,
#               "drink":drinks,
#               "pet":pets,
#               "cigar":cigars,
#              }
properties = [("nation",nationalities),
              ("color",colors),
              ("drink",drinks),
              ("pet",pets),
              ("cigar",cigars),
             ]

#prop_perms = {k:itertools.permutations(v, 5) for k,v in properties.items}

conditions = [
    # 8. The man living in the center house drinks milk.
    (frozenset(["drink"]), lambda assign:assign[2][2] == "milk"),
    # 9. The Norwegian lives in the first house.
    (frozenset(["nation"]),lambda assign:assign[0][0] == "Norwegian"),
    
    
    if assignment[2][2] != "milk":
        continue # 8. The man living in the center house drinks milk.
    if assignment[0][0] != "Norwegian":
        continue # 9. The Norwegian lives in the first house.
    
    if assignment[0].index("Brit") != assignment[1].index("red"):
        continue # 1. The Brit lives in the red house.
    if assignment[0].index("Swede") != assignment[3].index("dog"):
        continue # 2. The Swede keeps dogs as pets.
    if assignment[0].index("Dane") != assignment[2].index("tea"):
        continue # 3. The Dane drinks tea.
    if assignment[1].index("green") != assignment[1].index("white")-1:
        continue # 4. The green house is to the left of the white house.
    if assignment[1].index("green") != assignment[2].index("coffee"):
        continue # 5. The green homeowner drinks coffee.
    if assignment[4].index("Pall Mall") != assignment[3].index("bird"):
        continue # 6. The person who smokes Pall Mall rears birds.
    if assignment[1].index("yellow") != assignment[4].index("Dunhill"):
        continue # 7. The owner of the yellow house smokes Dunhill.
    
    # Conditions 8 and 9 moved to top for faster possibility elimination
    
    if abs(assignment[4].index("Blend") - assignment[3].index("cat")) != 1:
        continue # 10. The man who smokes Blend lives next to the one who keeps cats.
    if abs(assignment[4].index("Dunhill") - assignment[3].index("horse")) != 1:
        continue # 11. The man who keeps horses lives next to the man who smokes Dunhill.
    if assignment[4].index("Bluemaster") != assignment[2].index("beer"):
        continue # 12. The owner who smokes Bluemaster drinks beer.
    if assignment[0].index("German") != assignment[4].index("Prince"):
        continue # 13. The German smokes Prince.
    if abs(assignment[0].index("Norwegian") - assignment[1].index("blue")) != 1:
        continue # 14. The Norwegian lives next to the blue house.
    if abs(assignment[4].index("Blend") - assignment[2].index("water")) != 1:
        continue # 15. The man who smokes Blend has a neighbor who drinks water.
]

def loop(propname, prop, fixed_outer, free_inner):
    # It's a little bit funky to have propname and prop as separate
    # args when they always have to match, but I'm trusting myself
    # not to mess it up.
    assigned_properties = set([propname, *fixed_outer.keys()])
    checkable_conditions = [v for rr,cc in conditions if rr.issubset(assigned_properties)]
    
    for perm in itertools.permutations(prop, 5):
        assignment = fixed_outer | {propname:perm}
        if any([not cond(assignment) for cond in checkable_conditions]):
            continue

        remaining_props = free_inner.copy()
        next_prop,values = remaining_props.pop()
        loop(next_prop, values, assignment, remaining_props)
    

foo = properties.copy()
bar = foo.pop("nation")
loop("nation",bar,{},foo)

for n_perm in itertools.permutations(nationalities, 5):
    print(n_perm)
    if n_perm[0] != "Norwegian":
        continue # 9. The Norwegian lives in the first house.
        
    for d_perm in itertools.permutations(drinks, 5):
        if d_perm[2] != "milk":
            continue # 8. The man living in the center house drinks milk.
        if n_perm.index("Dane") != d_perm.index("tea"):
            continue # 3. The Dane drinks tea.
        
        for col_perm in itertools.permutations(colors, 5):
            if col_perm.index("green") != col_perm.index("white")-1:
                continue # 4. The green house is to the left of the white house.
            if n_perm.index("Brit") != col_perm.index("red"):
                continue # 1. The Brit lives in the red house.
            if col_perm.index("green") != d_perm.index("coffee"):
                continue # 5. The green homeowner drinks coffee.
            if abs(n_perm.index("Norwegian") - col_perm.index("blue")) != 1:
                continue # 14. The Norwegian lives next to the blue house.
                                    
            for p_perm in itertools.permutations(pets, 5):
                if n_perm.index("Swede") != p_perm.index("dog"):
                    continue # 2. The Swede keeps dogs as pets.
                    
                for cig_perm in itertools.permutations(cigars, 5):
                    #iter_count += 1
                    if cig_perm.index("Pall Mall") != p_perm.index("bird"):
                        continue # 6. The person who smokes Pall Mall rears birds.
                    if col_perm.index("yellow") != cig_perm.index("Dunhill"):
                        continue # 7. The owner of the yellow house smokes Dunhill.
 
                    if abs(cig_perm.index("Blend") - p_perm.index("cat")) != 1:
                        continue # 10. The man who smokes Blend lives next to the one who keeps cats.
                    if abs(cig_perm.index("Dunhill") - p_perm.index("horse")) != 1:
                        continue # 11. The man who keeps horses lives next to the man who smokes Dunhill.
                    if cig_perm.index("Bluemaster") != d_perm.index("beer"):
                        continue # 12. The owner who smokes Bluemaster drinks beer.
                    if n_perm.index("German") != cig_perm.index("Prince"):
                        continue # 13. The German smokes Prince.
                    
                    if abs(cig_perm.index("Blend") - d_perm.index("water")) != 1:
                        continue # 15. The man who smokes Blend has a neighbor who drinks water.
                    
                    print("Solution found!")
                    print(col_perm, n_perm, d_perm, cig_perm, p_perm)
                    print(f"Elapsed time: {time.time()- start_time} seconds")
print(f"Total time: {time.time()- start_time} seconds")

IndentationError: expected an indented block after 'for' statement on line 19 (1714465304.py, line 23)

In [17]:
import constraint

ModuleNotFoundError: No module named 'constraint'

A list of clues and the steps they're used in (in parentheses):
1. The Brit lives in the red house (9)
2. The Swede keeps dogs as pets (7,12,18)
3. The Dane drinks tea (10)
4. The green house is on the left of the white house (4)
5. The person who smokes Pall Malls rears birds (15,19,21)
6. The owner of the yellow house smokes Dunhill (5)
7. The green house’s owner drinks coffee (8)
8. The man living in the center house drinks milk (1)
9. The Norwegian lives in the first (leftmost) house (2)
10. The man who smokes Blends lives next to the one who keeps cats (15,23)
11. The man who keeps horses lives next to the man who smokes Dunhill (6)
12. The owner who smokes BlueMaster drinks beer (13,14,15,19,20,22)
13. The German smokes Princes (12,18)
14. The Norwegian lives next to the blue house (3)
15. The man who smokes Blends has a neighbor who drinks water (15,23)

Steps:

1. apply clue 8
2. apply clue 9
3. If the Norwegian is in the leftmost house, by clue 14 the blue house must be second.
4. By clue 4 the green and white houses must be next to one another, so house 1 cannot be green or white. It must be yellow. The last three houses are either RGW or GWR.
5. By clue 6, the first house has a Dunhill smoker.
6. By clue 11, the second house must have the horse.
7. By clue 2, the second house cannot have the Swede, and by clue 1 not the Brit, leaving only the Dane or German.
8. The final three houses must be RGW or GWR. The second is impossible because it places the green house with the milk drinker, when we know by clue 7 that the green house has a coffee drinker. Therefore assign the final three houses colors RGW.
9. By clue 1 the Brit must be in the third house.

|house|1|2|3|4|5|
|:----|:----|:----|:----|:----|:----|
|color|yellow|blue|red|green|white|
|Nationality|Norwegian|DG|Brit| | |
|cigar|Dunhill| | | | |
|drink| | |milk| | |
|pet| |horse| | | |


10. Assume per step 7 that the second house has the Dane. Then by clue 3 the second house has the tea drinker.

|house|1|2|3|4|5|
|:----|:----|:----|:----|:----|:----|
|color|yellow|blue|red|green|white|
|Nationality|Norwegian|Dane|Brit|
|cigar|Dunhill|Blen/Blue/Prince|Pal/Blen| | |
|drink| |tea|milk| | |
|pet| |horse| | | |


11. Assume that houses 4 and 5 have the Swede and German, respectively.
12. By clues 2 and 13, the fourth house has the dog, fifth the Prince smoker.
13. By clue 5 the second house cannot smoke Pall Mall, Dunhill is first, by clue 13 doesn't smoke Princes, and by clue 12 cannot smoke BlueMaster, so must smoke Blends.
14. The third house could smoke Pall Malls or BlueMaster, but clue 12 eliminates BlueMaster, leaving only Pall Malls. Assign BlueMaster to house 4.
15. By clue 5 the third house has the birds, by clue 12 the fourth house beer, by clue 10 the first house has cats, and by clue 15 the first house has water.
16. This leaves house 5 with coffee and fish.

|house|1|2|3|4|5|
|:----|:----|:----|:----|:----|:----|
|color|yellow|blue|red|green|white|
|Nationality|Norwegian|Dane|Brit|Swede|German|
|cigar|Dunhill|Blends|Pall Mall|BlueMaster|Prince|
|drink|water|tea|milk|beer|coffee|
|pet|cat|horse|bird|dog|fish|


17. Clue 7 has been violated. Backtrack to the latest assumption step, 11. Assume the opposite therefore; assume that houses 4 and 5 have the German and Swede, respectively.
18. By clue 13 the fourth house has the Prince smoker, by clue 2 the fifth house the dog.
19. The second house could smoke Pall Mall, Blends, or BlueMaster, but (clue 5) doesn't have birds so doesn't smoke Pall Mall, and clue 12 eliminates BlueMaster, leaving only Blends.
20. The third house could have the Pall Mall or BlueMaster smoker, but by clue 20 the BlueMaster smoker drinks beer, but we know the third house hsa the milk drinker, so this person must smoke Pall Mall. Assign BlueMaster to the fifth house.
21. By clue, 5 the third house must have birds.
22. By clue 12, the fifth house must drink beer.
23. By clue 10 house 1 must have cats; by clue 15 house 1 must have water.
24. By elimination, assign coffee and fish to house 4.

|house #|1|2|3|4|5|
|:----|:----|:----|:----|:----|:----|
|color|yellow|blue|red|green|white|
|Nationality|Norwegian|Dane|Brit|German|Swede|
|cigar|Dunhill|Blends|Pall Mall|Prince|BlueMaster|
|drink|water|tea|milk|coffee|beer|
|pet|cat|horse|bird|fish|dog|


All done!